In [1]:
import csv
import monpa
from monpa import utils
from collections import Counter
import pandas as pd
import numpy as np

+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.


c:\Users\emily\AppData\Local\Programs\Python\Python312\Lib\site-packages\monpa\__init__.py:142: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(checkpo

大立光新聞資料挑出

In [2]:
df = pd.read_csv("C:/Users/emily/Downloads/bda2025_mid_dataset/bda2025_202301-202503_內容數據_新聞.csv")
df.head(5)

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url
0,1672541649216_YahooNews,news,Yahoo新聞,股市匯市,2023-01-01 10:45:18,年關將近！準備換新鈔 央行「鈔前部署」3招辨真假,三立新聞網,記者戴玉翔／綜合報導新的一年即將到來，而在農曆年前，台灣人都有換新鈔的習慣，中央銀行預計會在...,https://tw.news.yahoo.com/%E5%B9%B4%E9%97%9C%E...
1,1672546740027_YahooNews,news,Yahoo新聞,股市匯市,2023-01-01 12:12:25,快加油！油價明調漲 汽油貴0.4元、柴油調降0.1元,王馨儀,台灣中油公司自明（2）日凌晨零時起汽油調漲0.4元、柴油調降0.1元，參考零售價格分別為92...,https://tw.news.yahoo.com/%E5%BF%AB%E5%8A%A0%E...
2,1672564766561_YahooNews,news,Yahoo新聞,股市匯市,2023-01-01 17:11:26,2023台股展望！農曆年還有紅包行情？,民視新聞網,民視新聞／綜合報導台北股市已經在2022年經歷過大熊市，那麼在即將到來的2023年，有沒有機...,https://tw.news.yahoo.com/2023%E5%8F%B0%E8%82%...
3,1672567021802_YahooNews,news,Yahoo新聞,股市匯市,2023-01-01 17:44:07,「反生命」喊話楊文科 要求終止生命園區開發,民視新聞網,民視新聞／ 綜合報導新竹縣的元旦升旗典禮，發生場外插曲，原來是新豐、湖口兩鄉居民，組成的反生...,https://tw.news.yahoo.com/%E5%8F%8D%E7%94%9F%E...
4,1672567849204_YahooNews,news,Yahoo新聞,股市匯市,2023-01-01 18:01:02,2023台股展望！農曆年還有紅包行情？,NaN,民視新聞／綜合報導台北股市已經在2022年經歷過大熊市，那麼在即將到來的2023年，有沒有機...,https://tw.news.yahoo.com/2023%E5%8F%B0%E8%82%...


In [3]:
#留title 跟content 就好
df_title_content_news = df[['post_time','title','content']]
df_title_content_news.head(5)

,post_time,title,content
0,2023-01-01 10:45:18,年關將近！準備換新鈔 央行「鈔前部署」3招辨真假,記者戴玉翔／綜合報導新的一年即將到來，而在農曆年前，台灣人都有換新鈔的習慣，中央銀行預計會在...
1,2023-01-01 12:12:25,快加油！油價明調漲 汽油貴0.4元、柴油調降0.1元,台灣中油公司自明（2）日凌晨零時起汽油調漲0.4元、柴油調降0.1元，參考零售價格分別為92...
2,2023-01-01 17:11:26,2023台股展望！農曆年還有紅包行情？,民視新聞／綜合報導台北股市已經在2022年經歷過大熊市，那麼在即將到來的2023年，有沒有機...
3,2023-01-01 17:44:07,「反生命」喊話楊文科 要求終止生命園區開發,民視新聞／ 綜合報導新竹縣的元旦升旗典禮，發生場外插曲，原來是新豐、湖口兩鄉居民，組成的反生...
4,2023-01-01 18:01:02,2023台股展望！農曆年還有紅包行情？,民視新聞／綜合報導台北股市已經在2022年經歷過大熊市，那麼在即將到來的2023年，有沒有機...


In [4]:
df_title_content_news.to_csv('df_title_content_output_news.csv', index=False, encoding='utf-8-sig')

In [5]:
#大立光相關關鍵字
keyword_list =['大立光','鏡頭' , '光學' ,'元件' ,'Largan','林恩平']

In [6]:
# 篩選 title 或 content 欄位中有任一關鍵字的資料
df_filtered_news = df_title_content_news[
    df_title_content_news.apply(
        lambda row: any(keyword in str(row['title']) or keyword in str(row['content']) for keyword in keyword_list),
        axis=1
    )
]
df_filtered_news.head(5)

,post_time,title,content
25,2023-01-03 13:50:25,台積電重返450元！台股強攻翻紅 指數漲86點收14224點,記者戴玉翔／台北報導台股今（3）日迎來 2023 年交易首日，開盤出師不利，盤中一度跌至 1...
44,2023-01-04 13:22:39,大立光及玉晶光雙雄領漲 光電類股大漲1％,光學雙雄-大立光（3008）及玉晶光（3406），今（4）日不畏蘋果被砍單、市值跌破2兆美元...
48,2023-01-04 14:09:22,台股終場小跌24點 光電族群續發威,[NOWnews今日新聞]美股2023年首個交易日4大指數皆走跌，台北股市今（4）日終場小跌...
50,2023-01-04 14:19:13,台股狹幅震盪小跌24點 人氣仍低迷,（中央社記者韓婷婷台北4日電）台股今天早盤雖一度受到美股收黑影響以低盤開出，但隨即在電子權值...
53,2023-01-04 15:05:13,台股多空交戰失守14200點 三大法人土洋對作賣超42.5億元,台股今 (4) 日在平盤上下遊走，光電族群表現強勢，其中，再生能源族群成為盤面多頭指標，不過...


In [7]:
df_filtered_news.shape

(17576, 3)

In [8]:
df_filtered_news.to_csv('df_largon_with_date_news.csv', index=False, encoding='utf-8-sig')

大立光ptt整理

In [9]:
df_ptt = pd.read_csv("C:/Users/emily/Downloads/bda2025_mid_dataset/bda2025_202301-202503_討論數據_ptt.csv")
df_ptt.head(5)

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url
0,1672502633844_PTT02R,bbs,Ptt,Stock,2023-01-01 00:02:20,Re: [心得] 2022，正二，0050，債券一次性比較,daze,"For what it's worth, 我的 guesstimation 是: 以加權...",http://www.ptt.cc/bbs/Stock/M.1672502560.A.07D...
1,1672504842040_PTT02R,bbs,Ptt,Stock,2023-01-01 00:39:10,[心得] 2022年報及一些小心得,d2623,跨年沒事就來發發年報，算是小紀念。 我現在的計算方式可能跟其他人不太相同，先附上圖。 h...,http://www.ptt.cc/bbs/Stock/M.1672504769.A.552...
2,1672505138779_PTT02R,bbs,Ptt,Stock,2023-01-01 00:44:17,[心得] 2022年報,seanlin10421,2022年初 https://i.imgur.com/KRNtGpw.png?1 2022...,http://www.ptt.cc/bbs/Stock/M.1672505060.A.971...
3,1672532512854_PTT02R,bbs,Ptt,Stock,2023-01-01 08:18:41,[新聞] 專訪／繁華過後 謝志堅看今年貨櫃船公司,Kingjames66,原文標題： 專訪／繁華過後 謝志堅看今年貨櫃船公司須有效控艙才能免虧 原文連結：htt...,http://www.ptt.cc/bbs/Stock/M.1672532323.A.9DB...
4,1672533240686_PTT02R,bbs,Ptt,Stock,2023-01-01 08:32:06,[心得] 2022年報 還是賠錢,Maxslack,想了很久年報要怎麼打，看了自己之前的年報跟半年報，整體形式上都流於流水帳，決 定換個方式來結...,http://www.ptt.cc/bbs/Stock/M.1672533128.A.BB2...


In [10]:
#留title 跟content 就好
df_title_content_ptt= df_ptt[['post_time','title','content']]
df_title_content_ptt.head(5)

,post_time,title,content
0,2023-01-01 00:02:20,Re: [心得] 2022，正二，0050，債券一次性比較,"For what it's worth, 我的 guesstimation 是: 以加權..."
1,2023-01-01 00:39:10,[心得] 2022年報及一些小心得,跨年沒事就來發發年報，算是小紀念。 我現在的計算方式可能跟其他人不太相同，先附上圖。 h...
2,2023-01-01 00:44:17,[心得] 2022年報,2022年初 https://i.imgur.com/KRNtGpw.png?1 2022...
3,2023-01-01 08:18:41,[新聞] 專訪／繁華過後 謝志堅看今年貨櫃船公司,原文標題： 專訪／繁華過後 謝志堅看今年貨櫃船公司須有效控艙才能免虧 原文連結：htt...
4,2023-01-01 08:32:06,[心得] 2022年報 還是賠錢,想了很久年報要怎麼打，看了自己之前的年報跟半年報，整體形式上都流於流水帳，決 定換個方式來結...


In [11]:
#大立光相關關鍵字
keyword_list =['大立光','鏡頭' , '光學' ,'元件' ,'Largan','林恩平']

In [12]:
# 篩選 title 或 content 欄位中有任一關鍵字的資料
df_filtered_ptt = df_title_content_ptt[
    df_title_content_ptt.apply(
        lambda row: any(keyword in str(row['title']) or keyword in str(row['content']) for keyword in keyword_list),
        axis=1
    )
]
df_filtered_ptt.head(5)

,post_time,title,content
48,2023-01-03 01:06:26,[創作] Ptt Alertor 股版追蹤排行榜,直接上圖https://i.imgur.com/0U9scpQ.png 原本想看一下2022...
68,2023-01-03 14:56:36,[情報] 0103 上市投信買賣超排行,1. 標題：投信買賣超排行 2. 來源： 富邦證券 3. 網址：https://fub...
112,2023-01-04 11:55:06,Re: [新聞] 輝達攜手鴻海 打造自駕汽車平台,之前有推文有聊到其他車廠自動駕駛狀態 像是Benz 目前許多車廠都是買Nvidia套裝或者軟...
138,2023-01-04 18:55:29,[新聞] 謝金河：機會總在絕望中誕生,原文標題：謝金河：機會總在絕望中誕生 原文連結：https://reurl.cc/GXRD...
176,2023-01-05 11:59:23,[新聞] 日經：戴爾計劃2024年全面停用大陸製造,原文標題： 日經：戴爾計劃2024年全面停用大陸製造晶片 原文連結： https://ud...


In [14]:
#因為發現創作跟情報的資訊都雜 因此只留[新聞]
df_news_only = df_filtered_ptt[df_filtered_ptt['title'].str.contains(r'\[新聞\]', na=False)]
df_news_only

,post_time,title,content
112,2023-01-04 11:55:06,Re: [新聞] 輝達攜手鴻海 打造自駕汽車平台,之前有推文有聊到其他車廠自動駕駛狀態 像是Benz 目前許多車廠都是買Nvidia套裝或者軟...
138,2023-01-04 18:55:29,[新聞] 謝金河：機會總在絕望中誕生,原文標題：謝金河：機會總在絕望中誕生 原文連結：https://reurl.cc/GXRD...
176,2023-01-05 11:59:23,[新聞] 日經：戴爾計劃2024年全面停用大陸製造,原文標題： 日經：戴爾計劃2024年全面停用大陸製造晶片 原文連結： https://ud...
220,2023-01-06 02:02:15,[新聞] 獨家》美企指控侵權 華碩、宏碁遭美調查,原文標題： 獨家》美企指控侵權 華碩、宏碁遭美調查 原文連結： https://ec...
225,2023-01-06 08:13:48,[新聞] HTC推新品！VIVE XR Elite登場 售價3490,原文標題： HTC推新品！VIVE XR Elite登場 售價34900元 原文連結： h...
...,...,...,...
58075,2025-02-22 16:26:05,[新聞] iPhone漲9％ 才能抵銷關稅壓力,iPhone漲9％ 才能抵銷關稅壓力 美銀潑冷水 低價策略失焦，多數蘋概股股價收黑 2025...
58148,2025-02-23 23:48:13,[新聞] 美股挫輝達跌逾4％ 法人：台股恐出現高檔,美股挫輝達跌逾4％ 法人：台股恐出現高檔震盪壓力 2025.02.22 16:53 ...
58166,2025-02-24 13:53:58,[新聞] 投資解密！去年國泰金砸重金布 AI 股 富,原文標題：投資解密！去年國泰金砸重金布 AI 股 富邦金縮手為哪樁？ 原文連結：http...
58351,2025-02-26 10:42:49,[新聞] 低軌衛星發射量增溫 法人看好這兩檔台系,原文標題： 低軌衛星發射量增溫 法人看好這兩檔台系供應鏈 原文連結： https://...


In [15]:
df_news_only.shape

(650, 3)

In [16]:
df_news_only.to_csv('df_largon_with_date_ppt.csv', index=False, encoding='utf-8-sig')